# Model Predictions (qubit_TransmonCross_cap_matrix)

## Configuration

In [1]:
# The parameter file is where the hyperparameters are set. 
# It's reccomended to look at that file first, its interesting and you can set stuff there

from parameters import *

## Library

In [2]:
# Disable some console warnings
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf# Disable some console warnings so you can be free of them printing. 
# Comment the next two lines if you are a professional and like looking at warnings.
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import os, gc
from pathlib import Path
import numpy as np
import pandas as pd
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model

## Dataset

### Load

In [3]:
# Load all of the nice data you saved from the previous notebook, or downloaded from the drive

if DATA_AUGMENTATION:
    if 'Try Both' not in ENCODING_TYPE:
        encoding = ENCODING_TYPE.replace(' ','_')
        X_train = np.load('{}/npy/x_train_{}_encoding_augmented.npy'.format(DATA_DIR, encoding), allow_pickle=True)
        X_val = np.load('{}/npy/x_val_{}_encoding_augmented.npy'.format(DATA_DIR, encoding), allow_pickle=True)
        X_test = np.load('{}/npy/x_test_{}_encoding_augmented.npy'.format(DATA_DIR, encoding), allow_pickle=True)
        y_train = np.load('{}/npy/y_train_{}_encoding_augmented.npy'.format(DATA_DIR, encoding), allow_pickle=True)
        y_val = np.load('{}/npy/y_val_{}_encoding_augmented.npy'.format(DATA_DIR, encoding), allow_pickle=True)
        y_test = np.load('{}/npy/y_test_{}_encoding_augmented.npy'.format(DATA_DIR, encoding), allow_pickle=True)
    
    elif 'Try Both' in ENCODING_TYPE:
        X_train_one_hot_encoding = np.load('{}/npy/x_train_one_hot_encoding_augmented.npy'.format(DATA_DIR), allow_pickle=True)
        X_val_one_hot_encoding = np.load('{}/npy/x_val_one_hot_encoding_augmented.npy'.format(DATA_DIR), allow_pickle=True)
        X_test_one_hot_encoding = np.load('{}/npy/x_test_one_hot_encoding_augmented.npy'.format(DATA_DIR), allow_pickle=True)
        y_train_one_hot_encoding = np.load('{}/npy/y_train_one_hot_encoding_augmented.npy'.format(DATA_DIR), allow_pickle=True)
        y_val_one_hot_encoding = np.load('{}/npy/y_val_one_hot_encoding_augmented.npy'.format(DATA_DIR), allow_pickle=True)
        y_test_one_hot_encoding = np.load('{}/npy/y_test_one_hot_encoding_augmented.npy'.format(DATA_DIR), allow_pickle=True)

        X_train_linear_encoding = np.load('{}/npy/x_train_linear_encoding_augmented.npy'.format(DATA_DIR), allow_pickle=True)
        X_val_linear_encoding = np.load('{}/npy/x_val_linear_encoding_augmented.npy'.format(DATA_DIR), allow_pickle=True)
        X_test_linear_encoding = np.load('{}/npy/x_test_linear_encoding_augmented.npy'.format(DATA_DIR), allow_pickle=True)
        y_train_linear_encoding = np.load('{}/npy/y_train_linear_encoding_augmented.npy'.format(DATA_DIR), allow_pickle=True)
        y_val_linear_encoding = np.load('{}/npy/y_val_linear_encoding_augmented.npy'.format(DATA_DIR), allow_pickle=True)
        y_test_linear_encoding = np.load('{}/npy/y_test_linear_encoding_augmented.npy'.format(DATA_DIR), allow_pickle=True)

else:
    if 'Try Both' not in ENCODING_TYPE:
        X_train = np.load('{}/npy/x_train_{}_encoding.npy'.format(DATA_DIR, encoding), allow_pickle=True)
        X_val = np.load('{}/npy/x_val_{}_encoding.npy'.format(DATA_DIR, encoding), allow_pickle=True)
        X_test = np.load('{}/npy/x_test_{}_encoding.npy'.format(DATA_DIR, encoding), allow_pickle=True)
        y_train = np.load('{}/npy/y_train_{}_encoding.npy'.format(DATA_DIR, encoding), allow_pickle=True)
        y_val = np.load('{}/npy/y_val_{}_encoding.npy'.format(DATA_DIR, encoding), allow_pickle=True)
        y_test = np.load('{}/npy/y_test_{}_encoding.npy'.format(DATA_DIR, encoding), allow_pickle=True)
    
    elif 'Try Both' in ENCODING_TYPE:
        X_train_one_hot_encoding = np.load('{}/npy/x_train_one_hot_encoding.npy'.format(DATA_DIR), allow_pickle=True)
        X_val_one_hot_encoding = np.load('{}/npy/x_val_one_hot_encoding.npy'.format(DATA_DIR), allow_pickle=True)
        X_test_one_hot_encoding = np.load('{}/npy/x_test_one_hot_encoding.npy'.format(DATA_DIR), allow_pickle=True)
        y_train_one_hot_encoding = np.load('{}/npy/y_train_one_hot_encoding.npy'.format(DATA_DIR), allow_pickle=True)
        y_val_one_hot_encoding = np.load('{}/npy/y_val_one_hot_encoding.npy'.format(DATA_DIR), allow_pickle=True)
        y_test_one_hot_encoding = np.load('{}/npy/y_test_one_hot_encoding.npy'.format(DATA_DIR), allow_pickle=True)

        X_train_linear_encoding = np.load('{}/npy/x_train_linear_encoding.npy'.format(DATA_DIR), allow_pickle=True)
        X_val_linear_encoding = np.load('{}/npy/x_val_linear_encoding.npy'.format(DATA_DIR), allow_pickle=True)
        X_test_linear_encoding = np.load('{}/npy/x_test_linear_encoding.npy'.format(DATA_DIR), allow_pickle=True)
        y_train_linear_encoding = np.load('{}/npy/y_train_linear_encoding.npy'.format(DATA_DIR), allow_pickle=True)
        y_val_linear_encoding = np.load('{}/npy/y_val_linear_encoding.npy'.format(DATA_DIR), allow_pickle=True)
        y_test_linear_encoding = np.load('{}/npy/y_test_linear_encoding.npy'.format(DATA_DIR), allow_pickle=True)


### Visualize

In [4]:
# Decide which model file & test set to use
chosen_path = "model/mlp_2_1000_3800_300_400_3_best_model.keras"      
X_test_cur = np.asarray(X_test)
y_test_cur = np.asarray(y_test)
y_encoding_format_name = encoding    

# Load y headers for labeling columns
y_headers_csv = f'y_characteristics_{y_encoding_format_name}_encoding.csv'
with open(y_headers_csv, 'r') as f:
    headers = f.readline().strip().split(',')

In [5]:
#run on CPU
tf.keras.backend.clear_session()
gc.collect()
try:
    tf.config.experimental.reset_memory_stats('GPU:0')
except Exception:
    pass

with tf.device('/CPU:0'):
    chosen_model = load_model(chosen_path, compile=False)  #dont compile it because we just need to predict
    y_pred = chosen_model.predict(X_test_cur, verbose=0)

print(f"\n—— {os.path.basename(chosen_path)} ——")
chosen_model.summary()
print(f"Samples: {len(X_test_cur)} | Targets dim: {y_test_cur.shape[1]}")


—— mlp_2_1000_3800_300_400_3_best_model.keras ——


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ fc0 (Dense)                     │ (None, 1000)           │         3,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_relu0 (LeakyReLU)         │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout0 (Dropout)              │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 3800)           │     3,803,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_relu1 (LeakyReLU)         │ (None, 3800)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout1 (Dropout)              │ (None, 3800)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 300)            │     1,140,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_relu2 (LeakyReLU)         │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout2 (Dropout)              │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc3 (Dense)                     │ (None, 400)            │       120,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_relu3 (LeakyReLU)         │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout3 (Dropout)              │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc_output (Dense)               │ (None, 3)              │         1,203 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,068,703 (19.34 MB)

 Trainable params: 5,068,703 (19.34 MB)

 Non-trainable params: 0 (0.00 B)

Samples: 291 | Targets dim: 3


# Scaled

In [6]:
#use a smaller view if you want
N_SAMPLES_TO_SHOW = 3

n_samples = min(N_SAMPLES_TO_SHOW, len(X_test_cur))
n_params  = y_test_cur.shape[1]

# scaled errors
sq_errors  = (y_test_cur - y_pred) ** 2
abs_errors = np.abs(y_test_cur - y_pred)

# scaled dataframe
rows = []
for i in range(n_samples):
    qubit_frequency_GHz, anharmonicity_MHz = X_test_cur[i, 0], X_test_cur[i, 1]
    for j in range(n_params):
        rows.append({
            "sample_idx": i,
            "qubit_frequency_GHz": qubit_frequency_GHz,
            "anharmonicity_MHz": anharmonicity_MHz,
            "param": headers[j],
            "ref":  y_test_cur[i, j],
            "pred": y_pred[i, j],
            "abs_error": abs_errors[i, j],
            "sq_error":  sq_errors[i, j],
        })
df = pd.DataFrame(rows)

# save scaled predictions
out_csv = Path(f"predictions_and_errors_{y_encoding_format_name}.csv")
df.to_csv(out_csv, index=False, float_format="%.6g")
print(f"\nSaved CSV -> {out_csv.resolve()}\n")

#pretty print per-sample (scaled)
for i in range(n_samples):
    sub = df[df["sample_idx"] == i].copy()
    sub = sub[["param", "ref", "pred", "abs_error", "sq_error"]]
    header_line = (
        f"— Sample {i} — "
        f"X: qubit_frequency_GHz={X_test_cur[i,0]:.9g}, anharmonicity_MHz={X_test_cur[i,1]:.9g}"
    )
    print(header_line)
    print(sub.to_string(index=False))
    print()

#print global stats (scaled)
print("Global scaled error stats:")
print("  min abs_error:", float(abs_errors.min()))
print("  median abs_error:", float(np.median(abs_errors)))
print("  max abs_error:", float(abs_errors.max()))
print("\nHere onehot/linear encoding and the MLP which maps categorical data to 1s and 0s is probably throwing off the global average. These will be rounded in the future and will probably always  round to the right number to reconstruct the correct category-- but for now it might throw off  the overall average error. In the future we might want to just have it consider the non-categorical data when finding an overall average and reporting that number.\n")


Saved CSV -> /home/abouzahr/ML_qubit_design/model_predict_qubit_TransmonCross_cap_matrix/predictions_and_errors_one_hot.csv

— Sample 0 — X: qubit_frequency_GHz=0.229409493, anharmonicity_MHz=0.858337674
                                                param      ref     pred  abs_error  sq_error
   design_options.connection_pads.readout.claw_length 0.333333 0.281250   0.052083  0.002713
design_options.connection_pads.readout.ground_spacing 0.152542 0.484131   0.331588  0.109951
                          design_options.cross_length 0.515152 0.543945   0.028794  0.000829

— Sample 1 — X: qubit_frequency_GHz=0.189596398, anharmonicity_MHz=0.886035915
                                                param      ref     pred  abs_error  sq_error
   design_options.connection_pads.readout.claw_length 0.090909 0.314209   0.223300  0.049863
design_options.connection_pads.readout.ground_spacing 1.000000 0.484863   0.515137  0.265366
                          design_options.cross_length 0.606061 0

# Unscaled

In [7]:
#load X feature names for the X scalers
with open('X_names', 'r') as f:
    X_index_names = f.read().splitlines()

#unscale X
X_test_unscaled = np.asarray(X_test_cur.copy())
for i in range(X_test_unscaled.shape[0]):
    for j in range(X_test_unscaled.shape[1]):
        scaler = joblib.load(f'scalers/scaler_X_{X_index_names[j]}.save')
        X_test_unscaled[i, j] = scaler.inverse_transform([[X_test_unscaled[i, j]]])[0][0]

#unscale y (refs and preds)
y_test_unscaled = np.asarray(y_test_cur.copy())
y_pred_unscaled = np.asarray(y_pred.copy())
for i in range(y_test_unscaled.shape[0]):
    for j in range(y_test_unscaled.shape[1]):
        scaler = joblib.load(f'scalers/scaler_y_{headers[j]}_{y_encoding_format_name}_encoding.save')
        y_test_unscaled[i, j] = scaler.inverse_transform([[y_test_unscaled[i, j]]])[0][0]
        y_pred_unscaled[i, j] = scaler.inverse_transform([[y_pred_unscaled[i, j]]])[0][0]

# Errors (unscaled)
sq_errors_unscaled  = (y_test_unscaled - y_pred_unscaled) ** 2
abs_errors_unscaled = np.abs(y_test_unscaled - y_pred_unscaled)

#build dataframe (unscaled)
rows_unscaled = []
for i in range(min(N_SAMPLES_TO_SHOW, len(X_test_unscaled))):
    qubit_frequency_GHz, anharmonicity_MHz = X_test_unscaled[i, 0], X_test_unscaled[i, 1]
    for j in range(n_params):
        rows_unscaled.append({
            "sample_idx": i,
           "qubit_frequency_GHz": qubit_frequency_GHz,
            "anharmonicity_MHz": anharmonicity_MHz,
            "param": headers[j],
            "ref_unscaled":  y_test_unscaled[i, j],
            "pred_unscaled": y_pred_unscaled[i, j],
            "abs_error_unscaled": abs_errors_unscaled[i, j],
            "sq_error_unscaled":  sq_errors_unscaled[i, j],
        })
df_unscaled = pd.DataFrame(rows_unscaled)

# save (unscaled)
out_csv_unscaled = Path(f"predictions_and_errors_unscaled_{y_encoding_format_name}.csv")
df_unscaled.to_csv(out_csv_unscaled, index=False, float_format="%.6g")
print(f"\nSaved CSV -> {out_csv_unscaled.resolve()}\n")

# Pretty print per-sample (unscaled)
for i in range(min(N_SAMPLES_TO_SHOW, len(X_test_unscaled))):
    sub = df_unscaled[df_unscaled["sample_idx"] == i].copy()
    sub = sub[["param", "ref_unscaled", "pred_unscaled", "abs_error_unscaled", "sq_error_unscaled"]]
    header_line = (
        f"— Sample {i} (Unscaled) — "
        f"X: qubit_frequency_GHz={X_test_unscaled[i,0]:.9g}, anharmonicity_MHz={X_test_unscaled[i,1]:.9g}"
    )
    print(header_line)
    print(sub.to_string(index=False))
    print()

#print global stats (unscaled)
print("Global unscaled error stats:")
print("  min abs_error:", float(abs_errors_unscaled.min()))
print("  median abs_error:", float(np.median(abs_errors_unscaled)))
print("  max abs_error:", float(abs_errors_unscaled.max()))
print("\nHere onehot/linear encoding and the MLP which maps categorical data to 1s and 0s is probably throwing off the global average. These will be rounded in the future and will probably always  round to the right number to reconstruct the correct category-- but for now it might throw off  the overall average error. In the future we might want to just have it consider the non-categorical data when finding an overall average and reporting that number.\n")



Saved CSV -> /home/abouzahr/ML_qubit_design/model_predict_qubit_TransmonCross_cap_matrix/predictions_and_errors_unscaled_one_hot.csv

— Sample 0 (Unscaled) — X: qubit_frequency_GHz=4.11507752, anharmonicity_MHz=-150.844392
                                                param  ref_unscaled  pred_unscaled  abs_error_unscaled  sq_error_unscaled
   design_options.connection_pads.readout.claw_length      0.000180       0.000163            0.000017       2.944694e-10
design_options.connection_pads.readout.ground_spacing      0.000005       0.000007            0.000002       3.895663e-12
                          design_options.cross_length      0.000260       0.000269            0.000009       8.860446e-11

— Sample 1 (Unscaled) — X: qubit_frequency_GHz=3.95948584, anharmonicity_MHz=-138.74412
                                                param  ref_unscaled  pred_unscaled  abs_error_unscaled  sq_error_unscaled
   design_options.connection_pads.readout.claw_length       0.00010       0.0